Predict Biased Word then use Marta's Features
------
We first use the bias detection module and then convert that word into a feature representation using Marta's features.

The features we use in this example are the ones proposed by Recasens et al, 2013 (https://nlp.stanford.edu/pubs/neutrality.pdf). For each word her features generate 32 features. Our labeling function then takes these features and applies a very simple regression (1 layer feed forward neural net).

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params

%load_ext autoreload
%autoreload 2

In [5]:
params = Params.read_params("experiment_params.json")

In [6]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dev_dataset = ExperimentDataset.init_dataset(params.dataset)

04/02/2020 06:38:51 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
100it [00:00, 4458.52it/s]


In [7]:
import pickle 
train_dataset = pickle.load(open(params.dataset["weakly_labeled_data"], "rb"))

In [8]:
train_dataset.add_data(train_dataset.get_val('weak_bias_label'), 'bias_label')

### Generating Features

In [9]:
from src.utils.shared_utils import get_bias_predictions 

In [10]:
bias_predictions_train = get_bias_predictions(train_dataset, params.intermediary_task, params.dataset, batch_size=32)

04/02/2020 06:38:58 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/02/2020 06:38:58 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/02/2020 06:38:58 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpka91dw1p
04/02/2020 06:39:02 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [11]:
bias_predictions_dev = get_bias_predictions(dev_dataset, params.intermediary_task, params.dataset, batch_size=32)

04/02/2020 06:41:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/02/2020 06:41:47 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/02/2020 06:41:47 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp0tqq8gda
04/02/2020 06:41:50 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [12]:
train_dataset.add_data(bias_predictions_train, "pre_tok_label_ids_predictions")

In [13]:
dev_dataset.add_data(bias_predictions_dev, "pre_tok_label_ids_predictions")

In [14]:
# importing the Featurizer created by Pryzant et al.
from src.utils.weak_labeling_utils import get_marta_featurizer, extract_marta_features

In [15]:
featurizer = get_marta_featurizer(params.dataset)

04/02/2020 06:43:24 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [16]:
marta_features_train = extract_marta_features(train_dataset, featurizer, bias_key="pre_tok_label_ids_predictions")

In [17]:
marta_features_dev = extract_marta_features(dev_dataset, featurizer, bias_key="pre_tok_label_ids_predictions")

In [19]:
marta_features_dev.shape #ensuring this is 90

torch.Size([81, 90])

In [20]:
train_dataset.add_data(marta_features_train, "marta_features")

In [21]:
dev_dataset.add_data(marta_features_dev, "marta_features")

### This is where the classification experiment starts

In [22]:
params = Params.read_params("experiment_params.json")

In [23]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [24]:
train_dataset.shuffle_data()
dev_dataset.shuffle_data()
train_dataloader = train_dataset.return_dataloader(batch_size=params.final_task['training_params']['batch_size'])
dev_dataloader = dev_dataset.return_dataloader(batch_size=32)

In [26]:
losses, evals = classification_experiment.train_model(train_dataloader, dev_dataloader, input_key="marta_features", label_key="bias_label", disable_tqdm=False)

In [27]:
evals

[[{'num_examples': 32, 'accuracy': 0.8125, 'auc': 0.7681159420289856},
  {'num_examples': 32, 'accuracy': 0.71875, 'auc': 0.8392156862745098},
  {'num_examples': 17, 'accuracy': 0.8235294117647058, 'auc': 0.875}],
 [{'num_examples': 32, 'accuracy': 0.8125, 'auc': 0.7681159420289856},
  {'num_examples': 32, 'accuracy': 0.71875, 'auc': 0.8352941176470589},
  {'num_examples': 17,
   'accuracy': 0.8235294117647058,
   'auc': 0.861111111111111}],
 [{'num_examples': 32, 'accuracy': 0.8125, 'auc': 0.7632850241545894},
  {'num_examples': 32, 'accuracy': 0.71875, 'auc': 0.8352941176470589},
  {'num_examples': 17,
   'accuracy': 0.8235294117647058,
   'auc': 0.861111111111111}],
 [{'num_examples': 32, 'accuracy': 0.78125, 'auc': 0.7584541062801933},
  {'num_examples': 32, 'accuracy': 0.71875, 'auc': 0.8352941176470589},
  {'num_examples': 17,
   'accuracy': 0.8235294117647058,
   'auc': 0.861111111111111}],
 [{'num_examples': 32, 'accuracy': 0.78125, 'auc': 0.7681159420289856},
  {'num_examples'

In [28]:
from src.utils.classification_utils import average_data

In [29]:
avg_evaluations = [average_data(epoch_evaluations) for epoch_evaluations in evals]

In [30]:
avg_evaluations

[{'num_examples': 81,
  'accuracy': 0.7777777777777778,
  'auc': 0.8186371864902698},
 {'num_examples': 81,
  'accuracy': 0.7777777777777778,
  'auc': 0.8141729728212631},
 {'num_examples': 81,
  'accuracy': 0.7777777777777778,
  'auc': 0.8122644620560695},
 {'num_examples': 81,
  'accuracy': 0.7654320987654321,
  'auc': 0.810355951290876},
 {'num_examples': 81,
  'accuracy': 0.7654320987654321,
  'auc': 0.8141729728212631}]